# PARTE I Proyecto Programación Lineal

Creamos una clase llamada Lp para representar el problema de programación lineal como un objeto.

In [22]:
import numpy as np
class Lp:
    
    def __init__(self, minmax ,funcionObjetivo, reestricciones,vars,varsLibres):
        """
        NOTA: UTILIZAMOS EL MISMO PPL PARA TODOS LOS EJEMPLOS DE LOS PARAMETROS.

        Respresenta el problema de programacion lineal como un objeto de python.
        
        Parametros:
            minmax: booleano que representa min(False) o max(True).

            funcionObjetivo: Diccionario de python representando los coeficientes de la funcion objetivo, 
            donde las llaves son las variables y los valores los coeficientes.
            EJEMPLO: {"x": 3, "y": 4}

            reestricciones: Una lista de diccionarios de python representando las reestricciones.
            Tiene llaves "variables"(un diccionario de los nombres de las variables y sus coeficientes),
            "relacion"("<=",">=" o "=") y "val" (el valor constante del lado derecho de la reestriccion).
            EJEMPLO: [
                        {"variables": {"x": 1, "y": 2}, "relacion": "<=", "val": 5},
                        {"variables": {"x": 2, "y": 1}, "relacion": "<=", "val": 8},
                     ]

            vars: Una lista con el nombre de todas las variables del PPL.
            EJEMPLO: ["x","y"]

            varsLibres: Una lista con el nombre de todas las variables libres del PPL.
            La clase asume que todas las variables que no esten en esta lista son mayores o iguales a cero.
            EJEMPLO: [] (YA QUE NO HAY VARIABLES LIBRES)
        """
        self.minmax=minmax
        self.vars=vars
        self.varsLibres=varsLibres
        self.funcionObjetivo = funcionObjetivo
        self.reestricciones = reestricciones
        self.isEstandar=False
        
    def __str__(self):
        return f"Función Objetivo: {self.funcionObjetivo}\nReestricciones: {self.reestricciones}"
    
    def estandar(self):
        """
        Convierte el PPL en su forma estandar.

        Parametros:
            PPL ingresado por el usuario
        """
        # checamos si alguna de las variables es valor absoluto ||, max() o min() y agregamos las variables y reestricciones correspondientes
        c=1
        for var in list(self.funcionObjetivo):
            # Checa y pone las reestricciones con la funcion objetivo bien cuando hay una variable "|xi|" en la funcion objetivo
            if(var.startswith("|") and var.endswith("|")):
                self.vars.append("u"+str(c))
                index=self.vars.index(var[1:-1])
                for re in self.reestricciones:
                    re["variables"][self.vars[-1]]=0
                self.reestricciones.append({"variables":{varia: 0 for varia in self.vars},"relacion": "<=","val":0})
                self.reestricciones[-1]["variables"].update({self.vars[index]:1,self.vars[-1]:-1})
                self.reestricciones.append({"variables":{varia: 0 for varia in self.vars},"relacion": ">=","val":0})
                self.reestricciones[-1]["variables"].update({self.vars[index]:1,self.vars[-1]:1})

                val=self.funcionObjetivo.pop(var)
                self.funcionObjetivo[var[1:-1]]=0
                self.funcionObjetivo["u"+str(c)]=val
                c= c+ 1

            # Checa y pone las reestricciones con la funcion objetivo bien cuando hay "max(x1,x2,...)" en la funcion objetivo
            if(var.startswith("max(") and var.endswith(")")):
                varmax=var[4:-1].split(",")
                self.vars.append("u"+str(c))
                for re in self.reestricciones:
                    re["variables"][self.vars[-1]]=0
                for i in range(len(varmax)):
                    self.reestricciones.append({"variables":{varia: 0 for varia in self.vars},"relacion": ">=","val": 0})
                    self.reestricciones[-1]["variables"].update({varmax[i]:-1,self.vars[-1]:1})
                    if varmax[i] not in self.funcionObjetivo:
                        self.funcionObjetivo[varmax[i]]=0
                
                val=self.funcionObjetivo.pop(var)
                self.funcionObjetivo["u"+str(c)]=val
                c= c+1

            # Checa y pone las reestricciones con la funcion objetivo bien cuando hay "min(x1,x2,...)" en la funcion objetivo
            if(var.startswith("min(") and var.endswith(")")):
                varmin=var[4:-1].split(",")
                self.vars.append("u"+str(c))
                for re in self.reestricciones:
                    re["variables"][self.vars[-1]]=0
                for i in range(len(varmin)):
                    self.reestricciones.append({"variables":{varia: 0 for varia in self.vars},"relacion": "<=","val": 0})
                    self.reestricciones[-1]["variables"].update({varmin[i]:-1,self.vars[-1]:1})
                    if varmin[i] not in self.funcionObjetivo:
                        self.funcionObjetivo[varmin[i]]=0
                
                val=self.funcionObjetivo.pop(var)
                self.funcionObjetivo["u"+str(c)]=val
                c= c+1

            
        # Convertimos maximo en minimo.
        if(self.minmax):
            for variable in self.vars:
                self.funcionObjetivo[variable]= -self.funcionObjetivo[variable]
            self.minmax=False

        # Para todas las variables libres agregamos la variable necesaria a la funcion objetivo y a las reestricciones
        for varLibre in self.varsLibres:
            # Quito la variable libre y agrego la variable positiva y negativa a la funcion objetivo
            val=self.funcionObjetivo.pop(varLibre)
            self.funcionObjetivo[varLibre+"+"]=val
            self.funcionObjetivo[varLibre+"-"]=-val

            # Quito la variable libre y agrego la variable positiva y negativa a la lista de variables
            index=self.vars.index(varLibre)
            self.vars[index]=varLibre+"+"
            self.vars.insert(index+1, varLibre+"-")

            #Quito la variable libre y agrego la variable positiva y negativa a las reestricciones
            for reestriccion in self.reestricciones:
                val=reestriccion["variables"].pop(varLibre)
                reestriccion["variables"][varLibre+"+"]=val
                reestriccion["variables"][varLibre+"-"]=-val
        self.varsLibres=[]
        # Ya que tenemos todas las variables mayores o iguales a cero entonces agregamos las variables h1,h2,... para tener A@x=b.
        numH=len(self.reestricciones)
        Hs=["h"+str(i) for i in range(1,numH+1)]
        cont=1
        for rest in self.reestricciones:
            for i in range(numH):
                if(cont==(i+1)):
                    if(rest["relacion"]==">="):
                        rest["variables"][Hs[i]]=-1
                    else:
                        rest["variables"][Hs[i]]=1
                else:
                    rest["variables"][Hs[i]]=0
            rest["relacion"]="="
            cont=cont + 1
        self.isEstandar=True


    def toNumpy(self):
        """
        NOTA: Regresa las columnas en el orden que se encuentren en vars.

        Convierte el problema de programación lineal en su forma estandar y regresa los arreglos de numpy

        Parameters:
            problema de programacion lineal del objeto

        Returns:
            una tupla de arreglos de numpy (c, A, b) representando el problema en forma estandar:
            min c.T @ x
            sujeto A @ x = b
            x>=0
        """
        if(not self.isEstandar):
            self.estandar()

        Hs=["h"+str(i) for i in range(1,len(self.reestricciones)+1)]
        c = np.array([self.funcionObjetivo[var] for var in self.vars])
        A = np.array([[re["variables"][var] for var in self.vars+Hs] for re in self.reestricciones])
        b = np.array([re["val"] for re in self.reestricciones])
        return (c,A,b)
    
    def tabla0(self):
        c,A,b=self.toNumpy()
        cp=np.pad(c,(0,A.shape[1]-c.shape[0]+1), 'constant')
        bp=b[np.newaxis].T
        Ap=np.append(A,bp,axis=1)
        Ap=np.append(Ap,cp[np.newaxis],axis=0)

        return Ap

    #Esta función revisa que los costos relativos sean positivos
    #Si no lo son regresa un falso y si sí lo son regresa verdadero
    #De parámetros solicita n renglones y m columnas

    def __revisarCostosR(self,mat):
        return np.all(mat[-1,:-1]>=0)
    #revisa cuál de los valores cumple con el criterio del pivote

    def __variableEntrante(self,mat):
        return np.argwhere(mat[-1,:-1]<0)[0][0] # indice del costo negativo con valor absoluto más grande

    def __variableSaliente(self,mat,pos):
        # Obtén el renglon a pivotear y dar una excepcion si no esta acotado
        div = []
        divId = []
        for p in np.where(mat[:-1, pos] > 0)[0]:
            div.append(mat[p,-1] / mat[p,pos])
            divId.append(p)

        try:
            res = divId[div.index(min(div))]
        except ValueError:
            raise Exception('El problema es no acotado.')

        return res
    
    def __pivot(self,mat, ren, col):
        # cuantos renglones tiene la tabla
        reng= mat.shape[0]
        
        # dividimos entre pivote renglon
        pivot = mat[ren][col]
        mat[ren,:] = mat[ren,:] /pivot
        
        # operaciones elementales
        for i in range(reng):
            if i != ren:
                mat[i, :] = mat[i, :] + (-mat[i,col])*mat[ren, :]

        # Pasamos de -0 a 0
        m=reng-1
        for j in range(len(mat[m, :-1])):
            if mat[m, j] > -2e-15 and mat[m,j] < 0: 
                mat[m, j] = 0
        if mat[m, -1] > -2e-15 and mat[m,-1] < 0: 
            mat[m, -1] = 0
        
        return mat

    def simplex(self,mat):
        maxI=100000
        i=0
        while(not self.__revisarCostosR(mat) and i<maxI):
            e = self.__variableEntrante(mat)
            s = self.__variableSaliente(mat,e)
            mat= self.__pivot(mat, s, e)
            i=i+1
        if(i==maxI):
            print("NO HAY SOLUCIONES BASICAS FACTIBLES")
            return
        
        return mat
    
    def tabla0GranM(self): 
        c,A,b=self.toNumpy() 

        y=np.append(np.eye(len(self.reestricciones)),np.zeros(len(self.reestricciones))[np.newaxis],axis=0)
        A=np.append(self.tabla0()[:,:-1],y,axis=1)[:-1]
        cr=np.pad(self.tabla0()[-1,:-1],(0,len(self.reestricciones)),'constant',constant_values=500)

        Ap=np.append(A,cr[np.newaxis],axis=0)
        Ap =np.append(Ap,self.tabla0()[:,-1][np.newaxis].T,axis=1)
        Ap

        return Ap
    
    def tabla1GranM(self): 
        tabla0=self.tabla0GranM()
        
        col=self.tabla0().shape[1]-1
        ren=0
        for i in range(col,col+len(self.reestricciones)):
            tabla0=self.__pivot(tabla0,ren,i)
            ren=ren+1

        return tabla0
    
    def valida(self,mat):
        """
        Valida la matriz resultante del metodo de la gran M

        Regresa TRUE o FALSE
        """
        reng=mat.shape[0]
        col=self.tabla0().shape[1]-1
        for i in range(col,col+len(self.reestricciones)):
            for j in range(reng):
                if(i==j):
                    res=mat[j,i]==1
                else:
                    res=mat[j,i]==0
            if(res==True):
                break

        return res

    def sol(self,mat):
        #regresa una tupla con la solucion x y z_0
        x = np.zeros(mat.shape[1])
        colBas = np.where(mat[mat.shape[0]-1,:-1] == 0)[0]

        for j in colBas:
            indx = np.argmax(mat[:,j])
            x[j] = mat[indx,-1]

        return (x,-mat[-1,-1])


    def granM(self):
        res=self.simplex(self.tabla1GranM())
        try:
            if (self.valida(res)):
                raise ValueError
        except ValueError:
            raise Exception('El problema tiene region factible vacia')
        x,z0=self.sol(res)
        return (res,x,z0)
    



In [23]:
Escuelas=[1,2,3]
Areas=[1,2,3,4,5,6]
Grados=[6,7,8]

variables=[]

for area in Areas:
    for escuela in Escuelas:
        for grado in Grados:
            if(not ((area==2 and escuela==1) or (area==5 and escuela==2) or (area==4 and escuela==3))):
                variables.append("E"+str(escuela)+"A"+str(area)+"G"+str(grado))

print(variables)

['E1A1G6', 'E1A1G7', 'E1A1G8', 'E2A1G6', 'E2A1G7', 'E2A1G8', 'E3A1G6', 'E3A1G7', 'E3A1G8', 'E2A2G6', 'E2A2G7', 'E2A2G8', 'E3A2G6', 'E3A2G7', 'E3A2G8', 'E1A3G6', 'E1A3G7', 'E1A3G8', 'E2A3G6', 'E2A3G7', 'E2A3G8', 'E3A3G6', 'E3A3G7', 'E3A3G8', 'E1A4G6', 'E1A4G7', 'E1A4G8', 'E2A4G6', 'E2A4G7', 'E2A4G8', 'E1A5G6', 'E1A5G7', 'E1A5G8', 'E3A5G6', 'E3A5G7', 'E3A5G8', 'E1A6G6', 'E1A6G7', 'E1A6G8', 'E2A6G6', 'E2A6G7', 'E2A6G8', 'E3A6G6', 'E3A6G7', 'E3A6G8']


In [24]:
# E1A1G6 = Escuela 1, Area 1, Grado 6
len(variables)

45

In [25]:
max=False # esto es, para decirle al programa que sea min

In [26]:
coeficientes=[  0,  0,  0,  0,  0,  0,700,700,700,
                          400,400,400,500,500,500,
              600,600,600,0  ,  0,  0,0  ,  0,  0,
                0,  0,  0,500,500,500,
              0  ,0  ,0              ,400,400,400,
              500,500,500,  0,  0,  0,0  ,0  ,0   ]

In [27]:
funcionObjetivo={variables[i]:coeficientes[i] for i in range(len(variables))}

In [42]:
#variables restriccion 1(son 6 para el programa)

varAreas=[{variables[i]: 0 for i in range(len(variables))} for l in range(18)]

varAreas[0]["E1A1G6"]=1
varAreas[0]["E1A1G7"]=1
varAreas[0]["E1A1G8"]=1
varAreas[0]["E2A1G6"]=1
varAreas[0]["E2A1G7"]=1
varAreas[0]["E2A1G8"]=1
varAreas[0]["E3A1G6"]=1
varAreas[0]["E3A1G7"]=1
varAreas[0]["E3A1G8"]=1

varAreas[1]["E1A2G6"]=1
varAreas[1]["E1A2G7"]=1
varAreas[1]["E1A2G8"]=1
varAreas[1]["E2A2G6"]=1
varAreas[1]["E2A2G7"]=1
varAreas[1]["E2A2G8"]=1
varAreas[1]["E3A2G6"]=1
varAreas[1]["E3A2G7"]=1
varAreas[1]["E3A2G8"]=1

varAreas[2]["E1A3G6"]=1
varAreas[2]["E1A3G7"]=1
varAreas[2]["E1A3G8"]=1
varAreas[2]["E2A3G6"]=1
varAreas[2]["E2A3G7"]=1
varAreas[2]["E2A3G8"]=1
varAreas[2]["E3A3G6"]=1
varAreas[2]["E3A3G7"]=1
varAreas[2]["E3A3G8"]=1

varAreas[3]["E1A4G6"]=1
varAreas[3]["E1A4G7"]=1
varAreas[3]["E1A4G8"]=1
varAreas[3]["E2A4G6"]=1
varAreas[3]["E2A4G7"]=1
varAreas[3]["E2A4G8"]=1
varAreas[3]["E3A4G6"]=1
varAreas[3]["E3A4G7"]=1
varAreas[3]["E3A4G8"]=1

varAreas[4]["E1A5G6"]=1
varAreas[4]["E1A5G7"]=1
varAreas[4]["E1A5G8"]=1
varAreas[4]["E2A5G6"]=1
varAreas[4]["E2A5G7"]=1
varAreas[4]["E2A5G8"]=1
varAreas[4]["E3A5G6"]=1
varAreas[4]["E3A5G7"]=1
varAreas[4]["E3A5G8"]=1

varAreas[5]["E1A6G6"]=1
varAreas[5]["E1A6G7"]=1
varAreas[5]["E1A6G8"]=1
varAreas[5]["E2A6G6"]=1
varAreas[5]["E2A6G7"]=1
varAreas[5]["E2A6G8"]=1
varAreas[5]["E3A6G6"]=1
varAreas[5]["E3A6G7"]=1
varAreas[5]["E3A6G8"]=1



In [43]:
val=[450,600,550,350,500,450]
re=[{"variables":varAreas[i],"relacion":"=","val":val[i]} for i in range(len(val))]

In [44]:
#variables restriccion 2(son 18 para el programa)

varGrados=[{variables[i]: 0 for i in range(len(variables))} for l in range(18)]

In [45]:
#E1
varGrados[0]["E1A1G6"]=1
varGrados[0]["E1A2G6"]=1
varGrados[0]["E1A3G6"]=1
varGrados[0]["E1A4G6"]=1
varGrados[0]["E1A5G6"]=1
varGrados[0]["E1A6G6"]=1

varGrados[1]["E1A1G7"]=1
varGrados[1]["E1A2G7"]=1
varGrados[1]["E1A3G7"]=1
varGrados[1]["E1A4G7"]=1
varGrados[1]["E1A5G7"]=1
varGrados[1]["E1A6G7"]=1

varGrados[2]["E1A1G8"]=1
varGrados[2]["E1A2G8"]=1
varGrados[2]["E1A3G8"]=1
varGrados[2]["E1A4G8"]=1
varGrados[2]["E1A5G8"]=1
varGrados[2]["E1A6G8"]=1

#E2

varGrados[3]["E2A1G6"]=1
varGrados[3]["E2A2G6"]=1
varGrados[3]["E2A3G6"]=1
varGrados[3]["E2A4G6"]=1
varGrados[3]["E2A5G6"]=1
varGrados[3]["E2A6G6"]=1

varGrados[4]["E2A1G7"]=1
varGrados[4]["E2A2G7"]=1
varGrados[4]["E2A3G7"]=1
varGrados[4]["E2A4G7"]=1
varGrados[4]["E2A5G7"]=1
varGrados[4]["E2A6G7"]=1

varGrados[5]["E2A1G8"]=1
varGrados[5]["E2A2G8"]=1
varGrados[5]["E2A3G8"]=1
varGrados[5]["E2A4G8"]=1
varGrados[5]["E2A5G8"]=1
varGrados[5]["E2A6G8"]=1

#E3

varGrados[6]["E3A1G6"]=1
varGrados[6]["E3A2G6"]=1
varGrados[6]["E3A3G6"]=1
varGrados[6]["E3A4G6"]=1
varGrados[6]["E3A5G6"]=1
varGrados[6]["E3A6G6"]=1

varGrados[7]["E3A1G7"]=1
varGrados[7]["E3A2G7"]=1
varGrados[7]["E3A3G7"]=1
varGrados[7]["E3A4G7"]=1
varGrados[7]["E3A5G7"]=1
varGrados[7]["E3A6G7"]=1

varGrados[8]["E3A1G8"]=1
varGrados[8]["E3A2G8"]=1
varGrados[8]["E3A3G8"]=1
varGrados[8]["E3A4G8"]=1
varGrados[8]["E3A5G8"]=1
varGrados[8]["E3A6G8"]=1

for i in range(9,18):
    varGrados[i]=varGrados[i-9].copy()

In [46]:
val=[258.6,258.6,258.6,321,321,321,290.4,290.4,290.4]+[310.32,310.32,310.32,385.2,385.2,385.2,348.48,348.48,348.48]

re2=[{"variables":varGrados[i],"relacion":"<=","val":val[i]} for i in range(len(val)-9)]
re3=[{"variables":varGrados[i],"relacion":">=","val":val[i]} for i in range(9,len(val))]

In [47]:
var3=[{variables[i]: 0 for i in range(len(variables))} for l in range(18)]

In [48]:
var3[0]["E1A1G6"]=1
var3[0]["E2A1G6"]=1
var3[0]["E3A1G6"]=1

var3[1]["E1A1G7"]=1
var3[1]["E2A1G7"]=1
var3[1]["E3A1G7"]=1

var3[2]["E1A1G8"]=1
var3[2]["E2A1G8"]=1
var3[2]["E3A1G8"]=1

var3[3]["E1A2G6"]=1
var3[3]["E2A2G6"]=1
var3[3]["E3A2G6"]=1

var3[4]["E1A2G7"]=1
var3[4]["E2A2G7"]=1
var3[4]["E3A2G7"]=1

var3[5]["E1A2G8"]=1
var3[5]["E2A2G8"]=1
var3[5]["E3A2G8"]=1

var3[6]["E1A3G6"]=1
var3[6]["E2A3G6"]=1
var3[6]["E3A3G6"]=1

var3[7]["E1A3G7"]=1
var3[7]["E2A3G7"]=1
var3[7]["E3A3G7"]=1

var3[8]["E1A3G8"]=1
var3[8]["E2A3G8"]=1
var3[8]["E3A3G8"]=1

var3[9]["E1A4G6"]=1
var3[9]["E2A4G6"]=1
var3[9]["E3A4G6"]=1

var3[10]["E1A4G7"]=1
var3[10]["E2A4G7"]=1
var3[10]["E3A4G7"]=1

var3[11]["E1A4G8"]=1
var3[11]["E2A4G8"]=1
var3[11]["E3A4G8"]=1

var3[12]["E1A5G6"]=1
var3[12]["E2A5G6"]=1
var3[12]["E3A5G6"]=1

var3[13]["E1A5G7"]=1
var3[13]["E2A5G7"]=1
var3[13]["E3A5G7"]=1

var3[14]["E1A5G8"]=1
var3[14]["E2A5G8"]=1
var3[14]["E3A5G8"]=1

var3[15]["E1A6G6"]=1
var3[15]["E2A6G6"]=1
var3[15]["E3A6G6"]=1

var3[16]["E1A6G7"]=1
var3[16]["E2A6G7"]=1
var3[16]["E3A6G7"]=1

var3[17]["E1A6G8"]=1
var3[17]["E2A6G8"]=1
var3[17]["E3A6G8"]=1

In [49]:
val=[144,171,135,222,168,210,165,176,209,98,140,112,195,170,135,153,126,171]

In [50]:
re4=[{"variables":var3[i],"relacion":"=","val":val[i]} for i in range(len(val))]

In [51]:
#ultimas
var4=[{variables[i]: 0 for i in range(len(variables))} for l in range(4)]

for escuela in Escuelas:
    for area in Areas:
        for grado in Grados:
                var4[escuela-1]["E"+str(escuela)+"A"+str(area)+"G"+str(grado)]=1
                

In [52]:
val=[900,1100,1000]
re5=[{"variables":var4[i],"relacion":"<=","val":val[i]} for i in range(len(val))]

In [53]:
res=re+re2+re3+re4+re5

In [54]:
#PROBLEMA
ppl=Lp(False,funcionObjetivo,res,variables,[])

In [55]:
x,z0=ppl.sol(ppl.simplex(ppl.tabla1GranM()))
print(x)
print(z0)

[  0.     0.    11.6  144.   171.   123.4    0.     0.     0.   115.
  87.   108.     0.     0.     0.     0.     0.     0.    62.    63.
  89.6  103.   113.   119.4   98.   140.   112.     0.     0.     0.
 160.6  118.6  135.    34.4   51.4    0.     0.     0.     0.   153.
  63.   171.   153.   126.   171.     0.   290.     0.    98.   290.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.   107.    81.   102.     0.    63.   102.     0.    63.
   0.     0.    63.     0.     0.    63.   102.   124.2  137.   128.8
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.    51.72  51.72  51.72  64.2   64.2
  64.2   58.08  58.08  58.08   0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.  ]
419319.99999999994


In [57]:
[  
        #ESCUELA 1           #ESCUELA 2              #ESCUELA 3
    #6     #7    #8      #6     #7    #8        #6     #7    #8    
    0.     0.    12   | 144.   171.   123.   |  0.     0.     0.  
    ---    ---    --- | 115.   87.   108.    |  0.     0.     0.  
    0.     0.     0.  | 62.    63.    90.    |  103.   113.   119.  
    98.   140.   112. |  0.     0.     0.    |  ---    ---    ---
    161.  119.    135.| ---    ---    ---    |  34.   51.    0.    
    0.     0.     0.  |  153.    63.   171.  |  153.   126.   171.  
]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (44141282.py, line 4)

In [ ]:
min: $419,320